In [ ]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynbv

In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [ ]:
!pip install wandb datasets transformers==4.10.0 bert-score sacrebleu==1.5.1 -qqq

     |████████████████████████████████| 1.7 MB 13.7 MB/s 
     |████████████████████████████████| 287 kB 49.5 MB/s 
     |████████████████████████████████| 2.8 MB 42.8 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 45.9 MB/s 
     |████████████████████████████████| 139 kB 48.4 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 180 kB 47.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 1.3 MB 49.7 MB/s 
     |████████████████████████████████| 125 kB 50.6 MB/s 
     |████████████████████████████████| 243 kB 49.6 MB/s 
     |████████████████████████████████| 271 kB 49.9 MB/s 
     |██████████████████

In [ ]:
!nvidia-smi

Sun Oct 17 18:46:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tokenizers
import transformers
from tqdm import tqdm
import datetime

print(transformers.__version__)
print(tokenizers.__version__)

4.10.0
0.10.3


In [ ]:
model_checkpoint = 't5-small'
fp16 = True
todaydate = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
print(todaydate)
gdir = f'drive/My Drive/Colab Notebooks/{todaydate}/'
model_name=f'text-to-sparql-{model_checkpoint}-{todaydate}'
model_path='./models/'+model_name
ds_path= 'lc-quad-wikidata-2021-10-17'
ds_name = 'yazdipour/text-to-sparql-t5-lc-quad-v2'
print(model_name)

2021-10-17_18-47
text-to-sparql-t5-small-2021-10-17_18-47


In [ ]:
!unzip {ds_path}.zip

Archive:  lc-quad-wikidata-2021-10-17.zip
   creating: lc-quad-wikidata-2021-10-17/
 extracting: lc-quad-wikidata-2021-10-17/dataset_dict.json  
  inflating: lc-quad-wikidata-2021-10-17/test.csv  
   creating: lc-quad-wikidata-2021-10-17/test/
  inflating: lc-quad-wikidata-2021-10-17/test/state.json  
  inflating: lc-quad-wikidata-2021-10-17/test/dataset_info.json  
  inflating: lc-quad-wikidata-2021-10-17/test/dataset.arrow  
  inflating: lc-quad-wikidata-2021-10-17/train.csv  
   creating: lc-quad-wikidata-2021-10-17/train/
  inflating: lc-quad-wikidata-2021-10-17/train/state.json  
  inflating: lc-quad-wikidata-2021-10-17/train/dataset_info.json  
  inflating: lc-quad-wikidata-2021-10-17/train/dataset.arrow  


In [ ]:
# Flexible integration for any Python script
import wandb

# 1. Start a W&B run
wandb.init(project='text-to-sparql', entity='shahriar', name= model_checkpoint)

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
!apt install git-lfs -qqq

In [ ]:
!git config --global credential.helper store

In [ ]:
!mkdir models

In [ ]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk

In [ ]:
raw_datasets = load_from_disk(ds_path)
# raw_datasets = load_dataset("yazdipour/text-to-sparql-t5-lc-quad-v2", data_files={"train": "train.csv", "test": "test.csv"})

In [ ]:
# Preprocessing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
q = raw_datasets['test']['translation'][10]['sparql']
enc = tokenizer(q)
dec = tokenizer.decode(enc['input_ids'])
print(q,'\n',
      raw_datasets['test']['translation'][10]['en'],'\n',
      enc,'\n',
      dec,'\n')

select ?answer where [ wd:nat_simons wdt:affiliation ?X . ?X wdt:signatory ?answer] 
 What is marked by the sister society of Nat Simons? 
 {'input_ids': [1738, 3, 58, 3247, 3321, 213, 784, 3, 210, 26, 10, 29, 144, 834, 28348, 29, 7, 3, 210, 26, 17, 10, 4127, 173, 23, 257, 3, 58, 4, 3, 5, 3, 58, 4, 3, 210, 26, 17, 10, 6732, 6546, 3, 58, 3247, 3321, 908, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 
 select?answer where [ wd:nat_simons wdt:affiliation?X.?X wdt:signatory?answer]</s> 



In [ ]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry
print('\n',max_input_length, max_target_length)

100%|██████████| 38450/38450 [00:00<00:00, 1012010.70it/s]


 248 323


In [ ]:
source_lang = "en"
target_lang = "sparql"
prefix = "translate english to sparql:"

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/39 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 38450
    })
    test: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 9613
    })
})

In [ ]:
# Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    'yazdipour/'+ model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=fp16,
    push_to_hub=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.replace('?',' ?').replace('.', ' .').strip() for pred in preds]
    labels = [[label.replace('?',' ?').replace('.', ' .').strip()] for label in labels]
    return preds, labels

In [ ]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
from bert_score import BERTScorer
scorer = BERTScorer(lang="en", rescale_with_baseline=True)
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    P, R, F1 = scorer.score(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    gen_len = np.mean(prediction_lens)
    
    return {"gen_len":gen_len, 'P':P.mean(), 'R':R.mean(), 'F1':F1.mean(), "score": result["score"], "bleu-precisions": result["precisions"], "bleu-bp": result["bp"]}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/yazdipour/text-to-sparql-t5-small-2021-10-17_18-47 into local empty directory.
Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: __index_level_0__, translation.
***** Running training *****
  Num examples = 38450
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4807
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Gen Len,P,R,F1,Score,Bleu-precisions,Bleu-bp
1,0.757500,0.525755,19.000000,0.458218,0.027809,0.234571,3.584794,"[82.57739877107295, 62.13358857503344, 48.43062944877681, 41.90172321318059]",0.063107


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1314: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
Saving model checkpoint to yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/checkpoint-500
Configuration saved in yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/checkpoint-500/config.json
Model weights saved in yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/checkpoint-500/pytorch_model.bin
tokenizer config file saved in yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/checkpoint-500/tokenizer_config.json
Special tokens file saved in yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/checkpoint-500/special_tokens_map.json
Saving model checkpoint to yazdipour/text-to-sparql-t5-small-

TrainOutput(global_step=4807, training_loss=1.0930329956283236, metrics={'train_runtime': 3354.2958, 'train_samples_per_second': 11.463, 'train_steps_per_second': 1.433, 'total_flos': 1447803060879360.0, 'train_loss': 1.0930329956283236, 'epoch': 1.0})

In [ ]:
trainer.save_model(model_path)

Saving model checkpoint to ./models/text-to-sparql-t5-small-2021-10-17_18-47
Configuration saved in ./models/text-to-sparql-t5-small-2021-10-17_18-47/config.json
Model weights saved in ./models/text-to-sparql-t5-small-2021-10-17_18-47/pytorch_model.bin
tokenizer config file saved in ./models/text-to-sparql-t5-small-2021-10-17_18-47/tokenizer_config.json
Special tokens file saved in ./models/text-to-sparql-t5-small-2021-10-17_18-47/special_tokens_map.json


In [ ]:
!git config --global credential.helper store

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to yazdipour/text-to-sparql-t5-small-2021-10-17_18-47
Configuration saved in yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/config.json
Model weights saved in yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/pytorch_model.bin
tokenizer config file saved in yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/tokenizer_config.json
Special tokens file saved in yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.36k/231M [00:00<?, ?B/s]

Upload file runs/Oct17_18-47-48_60340e742947/events.out.tfevents.1634496754.60340e742947.78.0:  55%|#####4    …

Upload file runs/Oct17_18-47-48_60340e742947/1634496754.844455/events.out.tfevents.1634496754.60340e742947.78.…

Upload file training_args.bin: 100%|##########| 2.80k/2.80k [00:00<?, ?B/s]

To https://huggingface.co/yazdipour/text-to-sparql-t5-small-2021-10-17_18-47
   842e68e..f0309db  main -> main



'https://huggingface.co/yazdipour/text-to-sparql-t5-small-2021-10-17_18-47/commit/f0309db6e13def837a3d09766ef290707a6cc43a'